Data Transformation & Cleaning

**Goal:** Convert hierarchical JSON data into relational tables and perform data cleaning.

**Input:** `mustika_rasa_full.json`  
**Outputs:** 
1. `df_recipes` (Recipe Metadata)
2. `df_ingredients` (Ingredient Details)

In [42]:
import pandas as pd
import json
import os

# 1. Setup Paths
BASE_DIR = os.getcwd()
INPUT_FILE = os.path.join(BASE_DIR, "mustika_rasa_full_cleaned.json")

# 2. Load JSON Data
try:
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    print(f"Successfully loaded {len(raw_data)} recipes.")
except FileNotFoundError:
    print("Error: JSON file not found. Please ensure 'mustika_rasa_full.json' is in this folder.")

Successfully loaded 1693 recipes.


## 1. Flattening the Data
We will transform the nested JSON into two flat lists to create our relational tables.

In [43]:
recipes_rows = []
ingredients_rows = []

ing_pk_counter = 1

for recipe in raw_data:
    # --- A. RECIPE TABLE ---
    rec_id = recipe.get('recipe_id')
    
    # Flatten instructions to single string
    instructions_clean = "\n".join(recipe.get('instructions', []) or [])
    
    # Keep tree structure as JSON string for reference
    ing_json_str = json.dumps(recipe.get('ingredient_groups', []), ensure_ascii=False)
    
    recipes_rows.append({
        'id': rec_id,
        'title_original': recipe.get('title_original'),
        'title_normalized': recipe.get('title_normalized'),
        'source_page': recipe.get('_source_page') or recipe.get('page_number'),
        'region': recipe.get('region'),
        'category': recipe.get('category'),
        'ingredient_json': ing_json_str,
        'instruction': instructions_clean
    })
    
    # --- B. INGREDIENTS TABLE ---
    groups = recipe.get('ingredient_groups', [])
    if groups:
        for group in groups:
            g_name = group.get('group_name', 'utama')
            
            for item in group.get('ingredients', []):
                ingredients_rows.append({
                    'id': f"ING_{str(ing_pk_counter).zfill(6)}",
                    'recipe_id': rec_id,
                    'ingredient_group': g_name,
                    'ingredient_original_name': item.get('item_original'),
                    'ingredient_normalized_name': item.get('item_normalized'),
                    'ingredient_quantity': item.get('quantity'),
                    'ingredient_unit': item.get('unit')
                })
                ing_pk_counter += 1

# Create DataFrames
df_recipes = pd.DataFrame(recipes_rows)
df_ingredients = pd.DataFrame(ingredients_rows)

print(f"Recipes Table: {df_recipes.shape}")
print(f"Ingredients Table: {df_ingredients.shape}")

Recipes Table: (1693, 8)
Ingredients Table: (13786, 7)


## 2. Cleaning Recipes Dataset

### 2.1 Clean Continues Page Recipes

In [44]:
# Inspect Recipes
display(df_recipes.head(5))
print(df_recipes.info())

,id,title_original,title_normalized,source_page,region,category,ingredient_json,instruction
0,MR_187_01,ARON,Aron,187,Tengger,Staple,"[{""group_name"": ""utama"", ""original_header"": ""B...","Bidji jagung direndam air 12 jam atau lebih, s..."
1,MR_187_02,AREM AREM,Arem Arem,187,NaN,Savory Snack,"[{""group_name"": ""kulit"", ""original_header"": ""B...","Beras dicuci, dikaru dan digarami.\nJika sudah..."
2,MR_188_01,AREM AREM ARON,Arem Arem Aron,188,NaN,Jajanan,"[{""group_name"": ""Bahan Utama (Aron)"", ""origina...",Aron direndam dengan garam selama ± 5 menit.\n...
3,MR_189_01,AREM AREM DJAGUNG,Arem Arem Jagung,189,NaN,Jajanan,"[{""group_name"": ""bahan utama"", ""original_heade...","Beras djagung ditjutji bersih, direndam 1 mala..."
4,MR_190_01,DJAGUNG BOSE,Jagung Bose,190,Timor,Makanan Pokok,"[{""group_name"": ""utama"", ""original_header"": ""B...",Jagung dicuci lalu direndam semalam dalam air....


<class 'pandas.DataFrame'>
RangeIndex: 1693 entries, 0 to 1692
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                1693 non-null   str  
 1   title_original    1693 non-null   str  
 2   title_normalized  1693 non-null   str  
 3   source_page       1693 non-null   int64
 4   region            976 non-null    str  
 5   category          1693 non-null   str  
 6   ingredient_json   1693 non-null   str  
 7   instruction       1693 non-null   str  
dtypes: int64(1), str(7)
memory usage: 105.9 KB
None


In [45]:
#check recipes title 
#this data is cleaned through manual edit on json file, ~ 22 recipes was edited manually
df_recipes[df_recipes['id']=='MR_221_01']

,id,title_original,title_normalized,source_page,region,category,ingredient_json,instruction


### 2.2 Clean Region

In [46]:
#check region 
region = df_recipes['region'].value_counts().reset_index()
region

,region,count
0,Rembang,78
1,Bali,76
2,Purwokerto,57
3,Palembang,51
4,Padang,50
...,...,...
99,Italia Utara (Serving Suggestion),1
100,Jalisco,1
101,Jawa Tengah - Jawa Timur,1
102,Purworedjo,1


In [47]:
region_map = {
    # --- JAWA TENGAH ---
    'Rembang': ('Rembang', 'Jawa Tengah'),
    'Purwokerto': ('Purwokerto', 'Jawa Tengah'),
    'Wonosobo': ('Wonosobo', 'Jawa Tengah'),
    'Banyumas': ('Banyumas', 'Jawa Tengah'),
    'Banjumas': ('Banyumas', 'Jawa Tengah'),
    'Bajumas': ('Banyumas', 'Jawa Tengah'),
    'Tegal': ('Tegal', 'Jawa Tengah'),
    'Solo': ('Solo (Surakarta)', 'Jawa Tengah'),
    'Jawa Tengah': ('Jawa Tengah', 'Jawa Tengah'),
    'Djawa Tengah': ('Jawa Tengah', 'Jawa Tengah'),
    'Brebes': ('Brebes', 'Jawa Tengah'),
    'Pati': ('Pati', 'Jawa Tengah'),
    'Kedu': ('Kedu', 'Jawa Tengah'),
    'Magelang': ('Magelang', 'Jawa Tengah'),
    'Cilacap (Tjilatjap)': ('Cilacap', 'Jawa Tengah'),
    'Cilacap': ('Cilacap', 'Jawa Tengah'),
    'Klaten': ('Klaten', 'Jawa Tengah'),
    'Purworedjo': ('Purworejo', 'Jawa Tengah'),
    
    # --- JAWA TIMUR ---
    'Madura': ('Madura', 'Jawa Timur'),
    'Malang': ('Malang', 'Jawa Timur'),
    'Madiun': ('Madiun', 'Jawa Timur'),
    'Jawa Timur': ('Jawa Timur', 'Jawa Timur'),
    'Surabaya': ('Surabaya', 'Jawa Timur'),
    'Pamekasan': ('Pamekasan', 'Jawa Timur'),
    'Magetan': ('Magetan', 'Jawa Timur'),
    'Sumenep': ('Sumenep', 'Jawa Timur'),
    'Tengger': ('Tengger', 'Jawa Timur'),
    'Sumberrejo': ('Sumberejo', 'Jawa Timur'), # Likely Bojonegoro area
    'Pacitan': ('Pacitan', 'Jawa Timur'),
    'Patjitan': ('Pacitan', 'Jawa Timur'),
    'Jawa Tengah - Jawa Timur': ('Jawa Tengah/Timur', 'Jawa Timur'), # Grouping to Jatim/Jateng border
    'Jawa Tengah/Timur': ('Jawa Tengah/Timur', 'Jawa Tengah/Timur'),
    'Sedayu': ('Sedayu', 'Jawa Timur'), # Assuming Gresik context, though Bantul exists

    # --- JAWA BARAT & BANTEN ---
    'Jawa Barat': ('Jawa Barat', 'Jawa Barat'),
    'Djawa Barat': ('Jawa Barat', 'Jawa Barat'),
    'Sukabumi': ('Sukabumi', 'Jawa Barat'),
    'Cianjur': ('Cianjur', 'Jawa Barat'),
    'Tjiandjur': ('Cianjur', 'Jawa Barat'),
    'Bandung': ('Bandung', 'Jawa Barat'),
    'Bogor': ('Bogor', 'Jawa Barat'),
    'Cirebon': ('Cirebon', 'Jawa Barat'),
    'Priangan': ('Priangan', 'Jawa Barat'),
    'Ciamis': ('Ciamis', 'Jawa Barat'),
    'Banten': ('Banten', 'Banten'), # Separated from Jabar for modern context

    # --- DKI JAKARTA ---
    'Jakarta': ('Jakarta', 'DKI Jakarta'),
    'Djakarta': ('Jakarta', 'DKI Jakarta'),
    'Pasarminggu': ('Pasar Minggu', 'DKI Jakarta'),

    # --- DIY YOGYAKARTA ---
    'Jogjakarta': ('Yogyakarta', 'DI Yogyakarta'),
    'Yogyakarta': ('Yogyakarta', 'DI Yogyakarta'),

    # --- BALI & NUSA TENGGARA ---
    'Bali': ('Bali', 'Bali'),
    'Sumbawa': ('Sumbawa', 'Nusa Tenggara Barat'),
    'Lombok': ('Lombok', 'Nusa Tenggara Barat'),
    'Timor': ('Timor', 'Nusa Tenggara Timur'),
    'Flores': ('Flores', 'Nusa Tenggara Timur'),

    # --- SUMATERA ---
    'Palembang': ('Palembang', 'Sumatera Selatan'),
    'Padang': ('Padang', 'Sumatera Barat'),
    'Sumatera Barat': ('Sumatera Barat', 'Sumatera Barat'),
    'Sumatera Barat: Singkarak': ('Singkarak', 'Sumatera Barat'),
    'Batak': ('Batak', 'Sumatera Utara'),
    'Tapanuli': ('Tapanuli', 'Sumatera Utara'),
    'Medan': ('Medan', 'Sumatera Utara'),
    'Atjeh': ('Aceh', 'Aceh'),
    'Aceh': ('Aceh', 'Aceh'),
    'Lampung': ('Lampung', 'Lampung'),
    'Riau': ('Riau', 'Riau'),
    'Duri': ('Duri', 'Riau'), # Assuming Duri Riau
    'Kotagadang': ('Koto Gadang', 'Sumatera Barat'),
    'Singkarak': ('Singkarak', 'Sumatera Barat'),
    'Minangkabau': ('Minangkabau', 'Sumatera Barat'),
    'Minang': ('Minangkabau', 'Sumatera Barat'),
    'Pariaman': ('Pariaman', 'Sumatera Barat'),
    'Bukittinggi': ('Bukittinggi', 'Sumatera Barat'),
    'Pajakumbuh': ('Payakumbuh', 'Sumatera Barat'),
    'Kajutanam': ('Kayu Tanam', 'Sumatera Barat'),
    'Sumatra': ('Sumatera', 'Sumatera'),
    'Kerinci': ('Kerinci', 'Jambi'),

    # --- KALIMANTAN ---
    'Bandjarmasin': ('Banjarmasin', 'Kalimantan Selatan'),
    'Banjarmasin': ('Banjarmasin', 'Kalimantan Selatan'),
    'Samarinda': ('Samarinda', 'Kalimantan Timur'),
    'Kalimantan': ('Kalimantan', 'Kalimantan'),

    # --- SULAWESI ---
    'Menado': ('Manado', 'Sulawesi Utara'),
    'Manado': ('Manado', 'Sulawesi Utara'),
    'Minahasa': ('Minahasa', 'Sulawesi Utara'),
    'Sulawesi Utara': ('Sulawesi Utara', 'Sulawesi Utara'),
    'Makasar': ('Makassar', 'Sulawesi Selatan'),
    'Makassar': ('Makassar', 'Sulawesi Selatan'),
    'Sulawesi Selatan': ('Sulawesi Selatan', 'Sulawesi Selatan'),
    'Bugis': ('Bugis', 'Sulawesi Selatan'),
    'Toraja': ('Toraja', 'Sulawesi Selatan'),
    'Toradja': ('Toraja', 'Sulawesi Selatan'),
    'Djeneponto': ('Jeneponto', 'Sulawesi Selatan'),
    'Palopo': ('Palopo', 'Sulawesi Selatan'),
    'Gorontalo': ('Gorontalo', 'Gorontalo'),
    'Mandar': ('Mandar', 'Sulawesi Barat'),
    'Buton': ('Buton', 'Sulawesi Tenggara'),
    'Poso': ('Poso', 'Sulawesi Tengah'),
    'Sulawesi Utara/Tengah': ('Sulawesi', 'Sulawesi'),

    # --- MALUKU & PAPUA ---
    'Irian Barat': ('Papua', 'Papua'),
    'Ambon': ('Ambon', 'Maluku'),
    'Maluku': ('Maluku', 'Maluku'),
    'Ternate': ('Ternate', 'Maluku Utara'),

    # --- OTHER / UNKNOWN / FOREIGN ---
    'Jawa': ('Jawa', 'Pulau Jawa'), # General
    'Italia Utara (Serving Suggestion)': ('Italia Utara', 'Luar Negeri'),
    'Jalisco': ('Jalisco', 'Luar Negeri'),
}

def map_region(raw_name):
    clean, prov = region_map.get(raw_name, (raw_name, 'Unknown'))
    return pd.Series([clean, prov])

df_recipes[['region_clean', 'province_group']] = df_recipes['region'].apply(map_region)

In [48]:
df_recipes['province_group'].value_counts()

province_group
Unknown                717
Jawa Tengah            291
Jawa Timur              98
Sumatera Barat          77
Bali                    76
Sulawesi Selatan        59
Jawa Barat              58
Sumatera Selatan        51
Kalimantan Selatan      40
Sumatera Utara          33
Sulawesi Utara          26
Nusa Tenggara Barat     21
Papua                   17
DI Yogyakarta           17
Banten                  16
Nusa Tenggara Timur     14
Maluku                  14
Aceh                    14
DKI Jakarta             13
Lampung                  6
Kalimantan Timur         5
Gorontalo                5
Sulawesi Barat           4
Kalimantan               4
Sulawesi Tenggara        3
Riau                     3
Pulau Jawa               3
Luar Negeri              2
Maluku Utara             1
Sumatera                 1
Sulawesi                 1
Jambi                    1
Sulawesi Tengah          1
Jawa Tengah/Timur        1
Name: count, dtype: int64

### 2.3 Clean Food Category

In [49]:
#take data from food_index 
food_index = pd.read_csv('food_index.csv')

# fixing sambal category
food_index[food_index['recipes_original_name'].str.lower().str.startswith('sambal') & (food_index['category'].str.lower() != 'SAMBAL SAMBALAN ')]['category'] = 'SAMBAL SAMBALAN'

display(food_index.head())

food_index['category'].value_counts()


/var/folders/lr/kb1ct1jn4kb5f9k6k7w4tjpw0000gp/T/ipykernel_22278/3087930836.py:5: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment.
Such chained assignment never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

Try using '.loc[row_indexer, col_indexer] = value' instead, to perform the assignment in a single step.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html#chained-assignment
  food_index[food_index['recipes_original_name'].str.lower().str.startswith('sambal') & (food_index['category'].str.lower() != 'SAMBAL SAMBALAN ')]['category'] = 'SAMBAL SAMBALAN'


,recipes_original_name,category
0,Sambal Goreng Kering,LAUK PAUK GORENGAN
1,Tempe,LAUK PAUK GORENGAN
2,Seng Geseng,LAUK PAUK GORENGAN
3,Serundeng,LAUK PAUK GORENGAN
4,Serundeng Ikan Mudjair,LAUK PAUK GORENGAN


category
DJADJANAN                        649
LAUK PAUK BASAH TIDAK BERKUAH    454
LAUK PAUK BASAH - BERKUAH        253
LAUK PAUK GORENGAN               125
LAUK PAUK BAKAR                   70
SAMBAL SAMBALAN                   64
MAKANAN UTAMA                     46
MINUMAN                           30
Name: count, dtype: int64

In [50]:
# Join index with recipes 

# 1. Rename the existing AI-generated category to avoid collision
df_recipes.rename(columns={'category': 'ai_category'}, inplace=True)

# 2. Create temporary lowercase columns for robust matching
df_recipes['join_key'] = df_recipes['title_original'].str.lower().str.strip()
food_index['join_key'] = food_index['recipes_original_name'].str.lower().str.strip()

# 3. Merge on the lowercase keys
df_recipes = df_recipes.merge(
    food_index[['join_key', 'category']], 
    on='join_key', 
    how='left'
)

# 4. Cleanup: Remove the temporary key
df_recipes.drop(columns=['join_key'], inplace=True)

In [51]:
#check value count of index

df_recipes['category'] = df_recipes['category'].fillna('Unknown')
df_recipes['category'].value_counts()

category
DJADJANAN                        593
LAUK PAUK BASAH TIDAK BERKUAH    423
LAUK PAUK BASAH - BERKUAH        224
Unknown                          135
LAUK PAUK GORENGAN               116
LAUK PAUK BAKAR                   71
SAMBAL SAMBALAN                   59
MAKANAN UTAMA                     45
MINUMAN                           27
Name: count, dtype: int64

In [52]:
# Mapping category using keywords from ai_category and title

# --- 1. Configuration ---

CATEGORIES = {
    'MAKANAN UTAMA': [
        'nasi', 'bubur', 'lontong', 'ketupat', 'tortilla', 'sagu', 'jagung', 'djagung', 'tiwul'
    ],
    'LAUK PAUK BASAH - BERKUAH': [
        'sayur', 'sajur', 'sop', 'soto', 'gulai', 'kare', 'kari', 'lodeh', 'asem', 
        'brongkos', 'rawon', 'semur', 'garang asam', 'gangan', 'pindang', 'santan'
    ],
    'LAUK PAUK BASAH TIDAK BERKUAH': [
        'pepes', 'botok', 'gadon', 'oseng', 'tumis', 'urap', 'pecel', 'petjel', 
        'karedok', 'gudeg', 'gudek', 'rendang', 'kalio', 'sambal goreng', 'sambel goreng', 
        'dendeng', 'terik', 'abon'
    ],
    'LAUK PAUK BAKAR': [
        'sate', 'saté', 'ayam bakar', 'grill', 'ikan bakar', 'panggang', 'klotok'
    ],
    'LAUK PAUK GORENGAN': [
        'goreng', 'perkedel', 'pekedel', 'dadar', 'martabak', 'lumpia', 'risoles', 
        'risolles', 'pastel', 'tahu', 'tempe', 'keripik', 'kerupuk', 'rempeyek', 'bakwan'
    ],
    'SAMBAL SAMBALAN': [
        'sambal', 'sambel', 'saos', 'bumbu', 'petis', 'dabu-dabu'
    ],
    'DJADJANAN': [
        'kue', 'kué', 'cake', 'bolu', 'jajanan', 'snack', 'lapis', 'desert', 'dessert', 
        'dodol', 'jenang', 'djenang', 'wajik', 'wadjid', 'getuk', 'gethuk', 'klepon', 
        'onde', 'apem', 'serabi', 'puding', 'poding', 'agar', 'kolak', 'pisang', 'ubi', 
        'singkong', 'tapai', 'tape', 'empek', 'pempek', 'tekwan', 'batagor', 'siomay'
    ],
    'MINUMAN': [
        'es ', 'minuman', 'wedang', 'bajigur', 'bandrek', 'camilan', 'sirup', 
        'jus', 'kopi', 'teh', 'cendol', 'tjendol', 'dawet'
    ]
}

# --- 2. Helper Functions ---

def get_category(text):
    """
    Maps input text to a category based on keywords.
    Returns 'Unknown' if no match is found or input is not a string.
    """
    if not isinstance(text, str):
        return "Unknown"
    
    text_lower = text.lower()
    
    for category, keywords in CATEGORIES.items():
        for key in keywords:
            # Check for exact word match (regex) OR substring match
            # Note: 'key in text' covers most cases, but regex handles boundaries better.
            if key in text_lower:
                return category
                
    return "Unknown"

def fill_missing_categories(df, source_column, target_column='category'):
    """
    Fills 'Unknown' values in the target_column using the source_column.
    """
    # Identify rows that need processing
    missing_mask = df[target_column] == 'Unknown'
    missing_count = missing_mask.sum()
    
    print(f"--- Processing source: '{source_column}' ---")
    print(f"Rows missing category before processing: {missing_count}")

    if missing_count == 0:
        print("No missing categories found. Skipping.")
        return

    # Extract source text for missing rows and map them
    source_data = df.loc[missing_mask, source_column]
    mapped_values = source_data.apply(get_category)

    # Update DataFrame only where we found a NEW valid category (not 'Unknown')
    # This prevents overwriting an 'Unknown' with another 'Unknown' unnecessarily
    found_mask = mapped_values != 'Unknown'
    
    if found_mask.sum() > 0:
        # We use the index from source_data to align the update correctly
        update_index = source_data[found_mask].index
        df.loc[update_index, target_column] = mapped_values[found_mask]

    # Reporting
    remaining_unknown = (df[target_column] == 'Unknown').sum()
    print(f"Successfully mapped: {found_mask.sum()}")
    print(f"Still Unknown: {remaining_unknown}")
    print("Processing complete.\n")

# --- 3. Execution Logic ---

# Ensure initial missing count is clear
initial_missing = (df_recipes['category'] == 'Unknown').sum()
print(f"Initial missing categories: {initial_missing}\n")

# Step 1: Map using 'ai_category'
fill_missing_categories(df_recipes, source_column='ai_category')

# Step 2: Map using 'title_original' (fallback for remaining unknowns)
fill_missing_categories(df_recipes, source_column='title_original')

# --- 4. Final Inspection ---
final_missing_mask = df_recipes['category'] == 'Unknown'
if final_missing_mask.sum() > 0:
    print("--- Sample of rows that are still Unknown ---")
    print(df_recipes.loc[final_missing_mask, ['title_original', 'ai_category', 'category']].head(5))
else:
    print("All categories have been successfully mapped!")

Initial missing categories: 135

--- Processing source: 'ai_category' ---
Rows missing category before processing: 135
Successfully mapped: 92
Still Unknown: 43
Processing complete.

--- Processing source: 'title_original' ---
Rows missing category before processing: 43
Successfully mapped: 31
Still Unknown: 12
Processing complete.

--- Sample of rows that are still Unknown ---
                  title_original ai_category category
301                     TUTURUGA   Main Dish  Unknown
302               TUTURUGA PENJU   Lauk Pauk  Unknown
337  ATJAR IKAN EMAS (Tjiandjur)   Lauk Pauk  Unknown
352                   BABI TJIEN        Lauk  Unknown
411       BUNDU DAGING (Makasar)   Main Dish  Unknown


In [53]:
#Manually Check the remain untidy data

missing_mask = df_recipes['category']=='Unknown'
df_recipes.loc[missing_mask]#[['title_original', 'title_normalized','ai_category','category']]

,id,title_original,title_normalized,source_page,region,ai_category,ingredient_json,instruction,region_clean,province_group,category
301,MR_365_02,TUTURUGA,Tuturuga,365,Menado,Main Dish,"[{""group_name"": ""utama"", ""original_header"": ""B...","Daging di-potong² ketjil.\nKentang dikupas, di...",Manado,Sulawesi Utara,Unknown
302,MR_366_02,TUTURUGA PENJU,Tuturuga Penyu,366,Menado,Lauk Pauk,"[{""group_name"": ""Bahan utama"", ""original_heade...",Daging penju dipotong-potong ketjil.\nBumbu² s...,Manado,Sulawesi Utara,Unknown
337,MR_389_01,ATJAR IKAN EMAS (Tjiandjur),Acar Ikan Mas (Cianjur),389,Cianjur,Lauk Pauk,"[{""group_name"": ""Bahan Utama"", ""original_heade...",Ikan emas dibersihkan. Kemudian diberi garam d...,Cianjur,Jawa Barat,Unknown
352,MR_396_02,BABI TJIEN,Babi Cien,396,NaN,Lauk,"[{""group_name"": ""utama"", ""original_header"": ""B...",Kentang dikuliti dengan diberi garam sedikit.\...,NaN,Unknown,Unknown
411,MR_432_02,BUNDU DAGING (Makasar),Bundu Daging (Makassar),432,Makassar,Main Dish,"[{""group_name"": ""utama"", ""original_header"": ""B...","Daging dipotong ketjil².\nKelapa diparut, dibu...",Makassar,Sulawesi Selatan,Unknown
412,MR_432_03,BUNDU MAKASAR,Bundu Makassar,432,Makassar,Main Dish,"[{""group_name"": ""utama"", ""original_header"": ""B...",Ajam ditjintjang sampai halus.\nBumbu² direnda...,Makassar,Sulawesi Selatan,Unknown
459,MR_463_01,HEE DJIE KEE,Hee Jie Kee,463,NaN,Lauk Pauk,"[{""group_name"": ""utama"", ""original_header"": ""B...",Dada ajam dipotong lebar tipis.\nHam dan dagin...,NaN,Unknown,Unknown
503,MR_491_02,LEMPPOK IKAN LAUT,Lempok Ikan Laut,491,Rembang,Ikan,"[{""group_name"": ""utama"", ""original_header"": ""B...",Nanas dan ikan dibersihkan di-potong².\nBumbu²...,Rembang,Jawa Tengah,Unknown
594,MR_544_01,PARAPE,Parape,544,Makasar,Lauk Pauk,"[{""group_name"": ""Bahan Utama"", ""original_heade...","Ikan dipotong-potong, dibersihkan.\nDigoreng s...",Makassar,Sulawesi Selatan,Unknown
782,MR_655_01,DANGKE\n(Duri),Dangke (Duri),655,Duri,Dairy,"[{""group_name"": ""utama"", ""original_header"": ""B...","Tangkai daun pepaja di-iris², diberi air ½ tja...",Duri,Riau,Unknown


In [54]:
#df_recipes[df_recipes['id']=='MR_221_01']
df_recipes.to_csv('df_recipes.csv')

# 3. Clean Ingredients

To do: 
1. Clean ingredient_normalized_name, the tail 
2. Clean ingredient_group --> how ingredient act at the recipes? 
2. Categorization of ingredient, modified from: Additive, Bakery, Beverages, Alcoholic Beverages, Cereals,
Dairy, Essential Oils, Fish, Flowers, Fruit, Fungi, Herbs, Legumes, Maize,
Meat, Nuts & Seeds, Plants, Seafood, Spices, Vegetables

In [ ]:
# Inspect Ingredients
display(df_ingredients.head(100))
print(df_ingredients.info())

,id,recipe_id,ingredient_group,ingredient_original_name,ingredient_normalized_name,ingredient_quantity,ingredient_unit
0,ING_000001,MR_187_01,utama,djagung putih pipilan,jagung putih pipilan,None,NaN
1,ING_000002,MR_187_02,kulit,beras,beras,1.0,liter
2,ING_000003,MR_187_02,kulit,garam,garam,1.0,sendok makan
3,ING_000004,MR_187_02,kulit,daun pisang batu,daun pisang batu,2.0,pelepah
4,ING_000005,MR_187_02,kulit,biting,biting,None,secukupnya
...,...,...,...,...,...,...,...
95,ING_000096,MR_199_01,utama,djagung,jagung,0.5,kg
96,ING_000097,MR_199_01,utama,nangka muda,nangka muda,0.5,buah
97,ING_000098,MR_199_02,utama,djagung,jagung,1.0,kg
98,ING_000099,MR_199_02,utama,ubi kaju,ubi kayu,2.0,kg


<class 'pandas.DataFrame'>
RangeIndex: 13818 entries, 0 to 13817
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          13818 non-null  str   
 1   recipe_id                   13818 non-null  str   
 2   ingredient_group            13818 non-null  str   
 3   ingredient_original_name    13818 non-null  str   
 4   ingredient_normalized_name  13818 non-null  str   
 5   ingredient_quantity         13189 non-null  object
 6   ingredient_unit             13608 non-null  str   
dtypes: object(1), str(6)
memory usage: 755.8+ KB
None


In [ ]:
counts = df_ingredients['ingredient_group'].str.lower().value_counts()

# Only show ingredients that appear 5 times or less
rare_ingredients = counts[counts <= 5] 
rare_ingredients


ingredient_group
kulit                                    5
bumbu tambahan                           5
alas nanas                               5
sebelah bawah                            5
bahan isi                                5
sambal                                   5
glazuur                                  5
bahan_instruksi_tambahan                 4
lain-lain                                4
adonan_laksan                            4
bumbu_saos                               4
tambahan (dari instruksi)                4
bumbu (lanjutan)                         4
bahan enten                              4
saos (bumbu halus)                       4
kepala (topping)                         4
bahan lain (diekstrak dari instruksi)    4
wadjud kelapa                            4
cream                                    4
keterangan_pelengkap                     3
bahan_utama_inferred                     3
pemanis                                  3
enten                                

In [ ]:
df_ingredients['ingredient_normalized_name'] = df_ingredients['ingredient_normalized_name'].str.lower()
df_ingredients['ingredient_normalized_name'].value_counts()

ingredient_normalized_name
garam                  1303
bawang merah            780
bawang putih            690
kelapa                  535
gula merah              416
                       ... 
sirup vanili              1
es parut                  1
es dan gula               1
biji sagu                 1
tape ketan/singkong       1
Name: count, Length: 1277, dtype: int64

In [ ]:
counts.to_csv('counts.csv')

In [ ]:
counts = df_ingredients['ingredient_original_name'].value_counts()

# Only show ingredients that appear 5 times or less
rare_ingredients = counts[counts <= 5] 

with pd.option_context('display.max_rows', None):
    print(rare_ingredients)

ingredient_original_name
beras djagung                                                        5
daun kelor                                                           5
daun turi                                                            5
asam sunti                                                           5
santan tjair                                                         5
buah melindjo                                                        5
telur itik                                                           5
daun mangkokan                                                       5
ikan teri                                                            5
udang ketjil                                                         5
daun so                                                              5
nanas                                                                5
ikan peda                                                            5
irisan bawang merah                                 

In [ ]:
import pandas as pd
import re

# Load your data (assuming it's already in a DataFrame named df)
# df = pd.read_csv('your_data.csv')

def clean_ingredient_name(text):
    if not isinstance(text, str):
        return ""
    
    # 1. Lowercase and strip whitespace
    text = text.lower().strip()
    
    # 2. Remove text inside parentheses (usually states/quantities like "1/2 tua" or "(iris)")
    text = re.sub(r'\([^)]*\)', '', text)
    
    # 3. Handle "/" - usually "minyak/mentega". We take the first item as primary.
    if '/' in text:
        text = text.split('/')[0]
        
    # 4. Remove specific noise characters and numbers
    text = re.sub(r'[0-9½¼\.,]', '', text) 
    
    # 5. OLD SPELLING NORMALIZATION (The "Ejaan Lama" Fixer)
    # Note: Order matters.
    replacements = {
        'oe': 'u',
        'dj': 'j',
        'tj': 'c',
        'nj': 'ny',
        'sj': 'sy',
        'ch': 'kh',
        'j': 'y' # Be careful: 'j' in old spelling was often 'y' sound, but 'dj' was 'j'. 
                  # In this specific dataset context, 'ajam' -> 'ayam' supports this rule.
    }
    
    # Apply replacements strictly (token-based or careful substring)
    # We iterate through the map. 
    for old, new in replacements.items():
        # Handle "ajam" -> "ayam", "minjak" -> "minyak"
        if old == 'j' and 'dj' in text: continue # Skip if part of 'dj'
        if old == 'j' and 'tj' in text: continue # Skip if part of 'tj'
        if old == 'j' and 'nj' in text: continue # Skip if part of 'nj'
        
        text = text.replace(old, new)

    # 6. DOMAIN SPECIFIC MAPPING (Synonyms & Prep Removal)
    # Map synonyms to a canonical name
    synonyms = {
        'brambang': 'bawang merah',
        'lombok': 'cabai',
        'cabe': 'cabai',
        'djae': 'jahe',
        'jae': 'jahe',
        'laos': 'lengkuas',
        'jeruk nipis': 'jeruk', # generalizing citrus if desired
        'jeruk purut': 'jeruk',
        'unclang': 'daun bawang',
        'vetsin': 'penyedap rasa',
        'micin': 'penyedap rasa'
    }
    
    # Remove preparation words
    stop_words = ['iris', 'halus', 'giling', 'potong', 'cincang', 'sisir', 'muda', 'tua', 'matang', 'segar']
    
    words = text.split()
    clean_words = []
    
    for w in words:
        if w in synonyms:
            clean_words.append(synonyms[w])
        elif w not in stop_words:
            clean_words.append(w)
            
    return ' '.join(clean_words).strip()

In [ ]:
df_ingredients

,id,recipe_id,ingredient_group,ingredient_original_name,ingredient_normalized_name,ingredient_quantity,ingredient_unit
0,ING_000001,MR_187_01,utama,djagung putih pipilan,jagung putih pipilan,None,NaN
1,ING_000002,MR_187_02,kulit,beras,beras,1.0,liter
2,ING_000003,MR_187_02,kulit,garam,garam,1.0,sendok makan
3,ING_000004,MR_187_02,kulit,daun pisang batu,daun pisang batu,2.0,pelepah
4,ING_000005,MR_187_02,kulit,biting,biting,None,secukupnya
...,...,...,...,...,...,...,...
13813,ING_013814,MR_1163_03,utama,kelapa,Kelapa,None,NaN
13814,ING_013815,MR_1163_03,utama,santan tjair,santan cair,3.0,gelas
13815,ING_013816,MR_1163_03,utama,santan kental,santan kental,1.0,gelas
13816,ING_013817,MR_1163_03,utama,kelapa muda,Kelapa muda,None,NaN


In [ ]:
# Apply the function

df_ingredients['ingredient_normalized_name'] = df_ingredients['ingredient_normalized_name'].apply(clean_ingredient_name)
counts = df_ingredients['ingredient_normalized_name'].value_counts()
counts.to_csv('counts.csv')
# Aggregation: Group by the new clean name and sum the counts
#df_clean = df_ingredients.groupby('ingredient_normalized_name')['count'].sum().sort_values(ascending=False).reset_index()

#print(df_clean.head(20))

## 3. Data Cleaning Section
Use this section to fix common OCR issues:
1. `None` values in titles.
2. Weird characters in units (e.g., "lt." vs "liter").
3. Standardize regions.

In [ ]:
# Example: Check for missing Normalized Titles
missing_titles = df_recipes[df_recipes['title_normalized'].isnull()]
print(f"Recipes with missing titles: {len(missing_titles)}")
missing_titles.head()

Recipes with missing titles: 0


,id,title_original,title_normalized,source_page,region,ai_category,ingredient_json,instruction,region_clean,province_group,category


In [ ]:
# Example: Normalize Units (Basic clean)
print("Top 20 Unique Units:")
print(df_ingredients['ingredient_unit'].value_counts().head(100))

# TODO: Add replacement logic here
# df_ingredients['ingredient_unit'] = df_ingredients['ingredient_unit'].replace({...})

Top 20 Unique Units:
ingredient_unit
sendok makan            1776
buah                    1749
sendok teh              1434
butir                   1109
kilogram                1045
                        ... 
rantang                    1
tunas                      1
sampai ikan terendam       1
juring                     1
kalo                       1
Name: count, Length: 100, dtype: int64


## 4. Export to CSV
Save the raw tables to disk before deep cleaning.

In [ ]:
df_recipes.to_csv("df_recipes.csv", index=False)
df_ingredients.to_csv("df_ingredient_recipes.csv", index=False)